In [6]:
#Python libraries
import numpy as np
from numpy.random import Generator, PCG64
from scipy import constants as sp
import time
import copy
import os

In [7]:
from modules.generate_quantum_states import generate_random_quantum_states_dataset 

In [8]:
cwd = os.getcwd()
sys.path.append(cwd+'/modules')

In [9]:
seed = int(time.time())
rng = Generator(PCG64(int(seed)))

In [12]:
outpath = cwd+'/input_data/100k/GHZ&W.txt'
N = 100000

In [13]:
 generate_random_quantum_states_dataset(N, rng, outpath, entanglement_type=['GHZ','W'], sep='\t')

# 1. Teoría

# 1.1. Sobre la generación de estados con entrelazamiento bipartito

Vamos a seguir una estrategia que se basa en la generación de estados de dos qubits que usamos en el estudio de $\mathcal{H}_2\otimes\mathcal{H}_2$.

Para un espacio, $\mathcal{H}$, de dimensión $N$, podemos tomar una base $\{|u_k\rangle ,\;\; k=1,...,N\equiv\text{dim}(\mathcal{H})\}$, de modo que $\forall|\psi\rangle\in\mathcal{H}$ se tiene:

>\begin{equation}
|\psi\rangle = \sum _{i=1}^{N} a_i|u_i\rangle \;\;\;\text{con}\;\;\;\sum |a_i|^2 = 1 \;\;\; a_i\in\mathbb{C}
\end{equation}

En general $a_j = r_j e^{i\phi _j}$ con $r_j>0$ y $\phi_j\in[0,2\pi)$. Para llevar a cabo la generación de estados aleatorios computacionalmente, vamos a considerar $0\leq r_j\leq 1$. Hemos de notar que considerar una cota superior para los módulos, $r_j$, no resta generalidad en tanto que es necesario normalizar el estado. Esto es, considerando $0\leq r_j\leq 1$ podemos samplear todo el espacio (salvo los estados separables, luego discutiremos porqué). Por ejemplo, podremos samplear un estado en que la amplitud asociada a uno de los estados base, $|u_k\rangle$ sea mucho mayor que la asociada al resto de estados. Para ello, simplemente tendrá que darse que para todos los estados $|u_j\rangle$ con $j\neq k$ se tenga $0\leq r_j<<r_k\leq 1$, de modo que al normalizar el estado por $\sqrt{\langle \psi|\psi\rangle}=\sqrt{\sum r_i ^2}$ se tendrá la relación de amplitudes deseada. De igual modo, podremos samplear estados en los que las amplitudes asociadas a todos los estados base sean muy parecidas, lo cual se tendrá cuando $r_j\simeq r_i$ $\forall i,j=1,...,N$.


Por otro lado, un estado separable se tendrá cuando $r_j=0$ $\forall j\neq k$. Puesto que para que se den este tipo de estados es necesario tener el 'cero' perfecto para los módulos de los coeficientes que acompañan a todos los estados base de $\mathcal{H}$ salvo a uno de ellos, podemos imaginarnos dicho estado como un punto (una variedad sin dimensión) sumergido en el espacio $\mathcal{H}$. Puesto que vamos a generar los módulos, $r_j$, y las fases, $\phi _j$, haciendo uso de un generador (numérico) de números aleatorios, nunca vamos a samplear el cero perfecto, por lo que realmente nunca vamos a tener un estado separable.

En el contexto del razonamiento que estábamos realizando hace dos párrafos, puedes comprobar que un estado $|\psi\rangle$ así construido (y normalizado) da lugar a una matriz densidad:

>\begin{equation}
|\psi\rangle\langle\psi| \doteq \frac{1}{\sum _i r_i ^2} \sum _{j,k} \tilde{c_{jk}} |u_j\rangle\langle u_k| \;\;\;\text{con}\;\;\; \tilde{c_{jk}} = r_jr_ke^{i(\phi _j -\phi _k})
\end{equation}

Así pues, la representación matricial de $|\psi\rangle\langle\psi|$ viene dada por:

>\begin{equation}
|\psi\rangle\langle\psi| \doteq \big(c_{jk}\big)_{j,k=1,...,N} \;\;\; \text{con}\;\;\; c_{jk} = \frac{\tilde{c_{jk}}}{\sum _i r_i ^2} = \frac{r_jr_ke^{i(\phi_j -\phi_k)}}{\sum _i r_i ^2}
\end{equation}

Esto es, $c_{jk}$ es el elemento de matriz que ocupa la fila $j$-ésima y la columna $k$-ésima. Hemos tomado implícitamente la base

>\begin{equation}
|u_k\rangle = \big(\delta _{ik}\big)_{i=1,...,N}
\end{equation}

Esto es, $|u_1\rangle^{T}=(1,0,...,0)$.

Si bien anteriormente nos interesaba estudiar la matriz densidad asociada al estado generado de $\mathcal{H}_2\otimes\mathcal{H}_2$, ahora vamos a trabajar con los vectores estado de $\mathcal{H}_2^{\otimes ^3}$. Sin embargo, antes de calcular dicho vector de estado he de cerciorarme de que el estado de dos qubits que he generado es realmente entrelazado. Para aplicar el criterio de entrelazamiento que venimos usando en $\mathcal{H}_2\otimes\mathcal{H}_2$ necesito su matriz densidad asociada (su producto exterior). Esto es, quiero generar estados de entrelazamiento bipartito, por tanto he de asegurarme de que el estado de dos qubits que he generado es realmente un estado entrelazado, antes de calcular su producto directo con el estado del tercer qubit. Así pues, el algoritmo a utilizar será:

1) Generar 4 números aleatorios $0\leq r_i\leq 1$, $i=1,...,N$.

2) Generar 4 números aleatorios $0\leq \phi_i< 2\pi$, $i=1,...,N$.

3) Construir la representación matricial $\big(c_{jk}\big)_{j,k=1,...,N}$ de acuerdo con la definición anterior.

4) Calcular su negatividad, $\lambda$. Si $\lambda >0$, entonces genero un estado de un qubit y calculo su producto directo con el estado de dos qubits entrelazado que he generado anteriormente.

5) Si $\lambda =0$, desecho el estado generado y vuelvo a 1). (Por motivos numéricos, esto no ocurrirá nunca.)

Además, la etiqueta del qubit que es separable del resto de qubits (1, 2 o 3) es escogida al azar en cada generación. Ahondaremos en esto en la propia función que implemente este algoritmo.